# Sentiment Analysis on IMDB Reviews using LSTM and Keras
created by Hans Michael
<hr>

### Steps
<ol type="1">
    <li>Load the dataset (50K IMDB Movie Review)</li>
    <li>Clean Dataset</li>
    <li>Encode Sentiments</li>
    <li>Split Dataset</li>
    <li>Tokenize and Pad/Truncate Reviews</li>
    <li>Build Architecture/Model</li>
    <li>Train and Test</li>
</ol>

<hr>
<i>Import all the libraries needed</i>

In [2]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

<hr>
<i>Preview dataset</i>

In [3]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


<hr>
<b>Stop Word</b> is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)

<i>Declaring the english stop words</i>

In [4]:
english_stops = set(stopwords.words('english'))

<hr>

### Load and Clean Dataset

In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in <b>load_dataset()</b> function, beside loading the dataset using <b>pandas</b>, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.

### Encode Sentiments
In the same function, I also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.

In [5]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


<hr>

### Split Dataset
In this work, I decided to split the data into 80% of Training and 20% of Testing set using <b>train_test_split</b> method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
39488    [this, saturday, halloween, year, today, time,...
13523    [a, slasher, flick, made, early, curse, anyone...
43327    [undoubtedly, funniest, movie, i, ever, seen, ...
22688    [what, point, film, it, totally, forgettable, ...
10380    [after, watching, movie, times, i, found, many...
                               ...                        
4638     [this, pretty, well, known, one, get, deep, th...
40762    [a, classic, movie, disney, reason, stopped, m...
18992    [it, funny, it, great, cast, give, great, perf...
44047    [andrew, gurland, cheats, fictionalized, true,...
23772    [it, funny, i, read, reviews, page, getting, m...
Name: review, Length: 40000, dtype: object 

23570    [i, know, film, went, unnoticed, long, i, saw,...
13235    [for, attention, chuck, davis, emefy, i, saw, ...
46924    [paulie, fantasy, littler, girl, perhaps, reco...
13890    [preston, waters, bad, summer, besides, birthd...
4263     [hitchcock, made, least, films, ordinary, man,...
 

<hr>
<i>Function for getting the maximum review length, by calculating the mean of all the reviews length (using <b>numpy.mean</b>)</i>

In [7]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

<hr>

### Tokenize and Pad/Truncate Reviews
A Neural Network only accepts numeric data, so we need to encode the reviews. I use <b>tensorflow.keras.preprocessing.text.Tokenizer</b> to encode the reviews into integers, where each unique word is automatically indexed (using <b>fit_on_texts</b> method) based on <b>x_train</b>. <br>
<b>x_train</b> and <b>x_test</b> is converted into integers using <b>texts_to_sequences</b> method.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using <b>tensorflow.keras.preprocessing.sequence.pad_sequences</b>.


<b>post</b>, pad or truncate the words in the back of a sentence<br>
<b>pre</b>, pad or truncate the words in front of a sentence

In [8]:
# ENCODE REVIEW
import dill
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)
with open('token.pkl', 'wb') as token_file:
    dill.dump(token, token_file)

print("tokens saved successfully!")

Encoded X Train
 [[    8  2243  1935 ...   239    26   514]
 [   39  1162   396 ...  1329   474    79]
 [ 3986  1488     3 ...     0     0     0]
 ...
 [    7    68     7 ...     0     0     0]
 [ 4085 45695  8938 ...     0     0     0]
 [    7    68     1 ...     0     0     0]] 

Encoded X Test
 [[    1    45     4 ...     0     0     0]
 [  208   590  2734 ...     0     0     0]
 [ 5976   939   150 ...     0     0     0]
 ...
 [   66     2 29414 ...     0     0     0]
 [  107    78     5 ...     0     0     0]
 [    1   244   177 ...     0     0     0]] 

Maximum review length:  130
tokens saved successfully!


<hr>

### Build Architecture/Model
<b>Embedding Layer</b>: in simple terms, it creates word vectors of each word in the <i>word_index</i> and group words that are related or have similar meaning by analyzing other words around them.

<b>LSTM Layer</b>: to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.
<ul>
    <li><b>Forget Gate</b>, decides information is to be kept or thrown away</li>
    <li><b>Input Gate</b>, updates cell state by passing previous output and current input into sigmoid activation function</li>
    <li><b>Cell State</b>, calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.</li>
    <li><b>Ouput Gate</b>, decides the next hidden state and used for predictions</li>
</ul>

<b>Dense Layer</b>: compute the input with the weight matrix and bias (optional), and using an activation function. I use <b>Sigmoid</b> activation function for this work because the output is only 0 or 1.

The optimizer is <b>Adam</b> and the loss function is <b>Binary Crossentropy</b> because again the output is only 0 and 1, which is a binary number.

In [9]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_shape = (max_length,)))
model.add(LSTM(LSTM_OUT))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

C:\Users\sy669\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 130, 32)             │       2,949,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,973,985 (11.34 MB)

 Trainable params: 2,973,985 (11.34 MB)

 Non-trainable params: 0 (0.00 B)

None


<hr>

### Training
For training, it is simple. We only need to fit our <b>x_train</b> (input) and <b>y_train</b> (output/label) data. For this training, I use a mini-batch learning method with a <b>batch_size</b> of <i>128</i> and <i>5</i> <b>epochs</b>.

Also, I added a callback called **checkpoint** to save the model locally for every epoch if its accuracy improved from the previous epoch.

In [10]:
checkpoint_train = ModelCheckpoint(
    'models/LSTM_train.keras',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [11]:
model.fit(x_train, y_train, batch_size = 128, epochs =10, callbacks=[checkpoint_train])

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.5170 - loss: 0.6918
Epoch 1: accuracy improved from -inf to 0.54588, saving model to models/LSTM_train.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 51s 151ms/step - accuracy: 0.5171 - loss: 0.6918
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.6297 - loss: 0.6467
Epoch 2: accuracy improved from 0.54588 to 0.63475, saving model to models/LSTM_train.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 48s 153ms/step - accuracy: 0.6297 - loss: 0.6467
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.6328 - loss: 0.6129
Epoch 3: accuracy improved from 0.63475 to 0.65482, saving model to models/LSTM_train.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 48s 154ms/step - accuracy: 0.6329 - loss: 0.6129
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.7070 - loss: 0.5949
Epoch 4: accuracy improved from 0.65482 to 0.70630, saving model to models/LSTM_train.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 48s 154ms/step - acc

<hr>

### Testing
To evaluate the model, we need to predict the sentiment using our <b>x_test</b> data and comparing the predictions with <b>y_test</b> (expected output) data. Then, we calculate the accuracy of the model by dividing numbers of correct prediction with the total data. Resulted an accuracy of <b>86.63%</b>

In [12]:
# Get predicted probabilities
y_pred_probs = model.predict(x_test, batch_size=128)

# Convert probabilities to class labels (0 or 1)
y_pred = (y_pred_probs > 0.5).astype(int)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {:.2f}%'.format((true / len(y_pred)) * 100))


79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step
Correct Prediction: 8387
Wrong Prediction: 1613
Accuracy: 83.87%


---

### Load Saved Model

Load saved model and use it to predict a movie review statement's sentiment (positive or negative).

In [13]:
loaded_model = load_model('models/LSTM.keras')

Receives a review as an input to be predicted

In [25]:
review = str(input('Movie Review: '))

Movie Review:  la la land vibrant homage classic musicals blending romance ambition backdrop los angeles ryan gosling emma stone deliver enchanting performances aspiring artist musician films cinematography music choreography create mesmerizing experience beautifully crafted tale dreams love sacrifices make


The input must be pre processed before it is passed to the model to be predicted

In [30]:
# Pre-process input
def preprocessing(review):
    english_stops = set(stopwords.words('english'))
    max_length=130
    regex = re.compile(r'[^a-zA-Z\s]')
    review = regex.sub('', review)
    
    words = review.split(' ')
    filtered = [w for w in words if w not in english_stops]
    filtered = ' '.join(filtered)
    filtered = [filtered.lower()]
    print('Filtered: ', filtered)
    tokenize_words = token.texts_to_sequences(filtered)
    tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
    return tokenize_words
tokenize_words=preprocessing(review)    

Filtered:  ['la la land vibrant homage classic musicals blending romance ambition backdrop los angeles ryan gosling emma stone deliver enchanting performances aspiring artist musician films cinematography music choreography create mesmerizing experience beautifully crafted tale dreams love sacrifices make']


Once again, we need to tokenize and encode the words. I use the tokenizer which was previously declared because we want to encode the words based on words that are known by the model.

This is the result of the prediction which shows the **confidence score** of the review statement.

In [16]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[[0.99333644]]


If the confidence score is close to 0, then the statement is **negative**. On the other hand, if the confidence score is close to 1, then the statement is **positive**. I use a threshold of **0.7** to determine which confidence score is positive and negative, so if it is equal or greater than 0.7, it is **positive** and if it is less than 0.7, it is **negative**

In [17]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive


In [18]:
import dill
with open('model.pkl', 'wb') as model_file:
    dill.dump(loaded_model, model_file)

print("Model saved successfully!")


Model saved successfully!


In [31]:
with open('preprocess.pkl', 'wb') as model_file:
    dill.dump(preprocessing, model_file)

print("preprocess saved successfully!")

preprocess saved successfully!


In [20]:
with open('model.pkl', 'rb') as file:
    loaded_model = dill.load(file)

In [21]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[[0.99333644]]


In [26]:
with open('preprocess.pkl', 'rb') as file:
    pre_model = dill.load(file)

In [27]:
tokenize_words=pre_model(review)

Filtered:  ['la la land vibrant homage classic musicals blending romance ambition backdrop los angeles ryan gosling emma stone deliver enchanting performances aspiring artist musician films cinematography music choreography create mesmerizing experience beautifully crafted tale dreams love sacrifices make']
[[ 981  981 1265 5521 2984  270 3019 9413  781 5198 3359 3061 3706 1539
  8981 3377 1679 1535 7765  272 5674 1593 4959   35  539  116 4102  918
  6138  463 1183 2815  684 1345   40 8055   25    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [28]:
import pickle
with open('model.pkl','rb') as m:
    model=pickle.load(m)

In [29]:
model.predict(tokenize_words)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


array([[0.99333644]], dtype=float32)